# Scripts for learning 

In [86]:
dimensions_ingredients = 702
dimensions_nutrition_facts = 22
number_of_clusters=100
import json

In [31]:
# Some Imports
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD
import sklearn.metrics as sm
import numpy as np
import json
import pickle
from scipy.stats import pearsonr

In [3]:
# Define Inputs
X = None
Y = None

# Load Recipes
filename="recipes.json"
with open(filename) as json_data:
    recipes = json.load(json_data)
    Y = list(recipes.keys())
    X = np.zeros([len(Y), dimensions_ingredients+dimensions_nutrition_facts])
    index = 0
    for label, recipe in recipes.items():
        for ingredient in recipe["ingredients"]:
            X[index, ingredient[0]] = ingredient[1]
        for fact in recipe["nutrition"]:
            X[ index, dimensions_ingredients + fact[0] ] = fact[1]
        index+=1
print("File "+filename+" was successfully read")

File recipes.json was successfully read


In [4]:
def removeOutlayers(X,Y,thr=45.0):
    X_new = []
    Y_new = []
    mtrs = [(np.mean(xi), np.std(xi)) for xi in X.T]
    for xi,yi in zip(X,Y):
        out = False
        for j,f in enumerate(xi):
            if f > thr*mtrs[j][1]:
                out = True
        if not out:
            X_new.append(xi)
            Y_new.append(yi)
    return np.asarray(X_new), Y_new
    

X_new, Y_new = removeOutlayers(X,Y)

print('Original observations', X.shape)
print('After removing outlayers', X_new.shape)
        

Original observations (18215, 724)
After removing outlayers (17273, 724)


In [5]:
X = X_new
Y = Y_new

In [6]:
# See one example
print(Y[0])
print(X[0,:])

Salt Cod with Garlic Cream
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00

In [7]:
# Train K-Means
# model = KMeans(n_clusters=number_of_clusters)
# model.fit(X)

In [8]:
# Show the labels
# model.labels_

In [9]:
# Show one of the groups
groups = []
for group in range(0, number_of_clusters):
    groups.append([])
#for index in range(0,len(Y)):
    # groups[model.labels_[index]].append(Y[index])

# Now go with nearest neighbors

In [10]:
X_ingredients = X[:,0:dimensions_ingredients]

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X_ingredients)

In [11]:
# Find the nearest neighbor to a given one
def _neighbors_of(X):
    X_=np.mean(X, axis=0)
    X_ = X_.reshape(1,-1)
    neighbors=[]
    distances, indices = nbrs.kneighbors(X_)
    for n_index in indices[0]:
        neighbors.append(Y[int(n_index)])
    return neighbors
    
def neighbors_of(index):
    X=X_ingredients[index:index+1,:]
    return _neighbors_of(X)
    
neighbors_of(100)

['Pork loin with Parma ham and oregano',
 'The Chavez Family Christmas Eve Carne Enchilada',
 'Indoor Pulled Pork',
 'Thai Style Grilled Pork Neck',
 'Chinese Roast Pork',
 'Herb-Crusted Pork Roast',
 "Cynthia's Spicy Shredded Pork",
 'Bone-In Pork Loin With Dijon Sauce',
 'Mustard, Maple, Horseradish Pork',
 'Portuguese Spicy Pulled Pork Sandwiches SundaySupper']

In [12]:
# Or find the nearest to a group of those
def neighbors_of_list(elements, cluster=False):
    X=X_ingredients[elements,:]
    if cluster:
        print("Not implemented")
    else:
        return _neighbors_of(X)
        
neighbors_of_list([0, 268, 3930, 100])

['Easy Homemade Mexican Chorizo',
 'Pork Dim Sum',
 'Pork Loin Braised with Cabbage',
 'Harissa Pork With Herbed Couscous',
 'Fish-Flavored Pork',
 'Grilled Sardines with Piquillo Pepper Sauce',
 'Fried Halloumi Cheese',
 'Chopped Tomato Salad',
 'Double Duty Tomatoes: Zesty Tomato Water and Roasted Tomato Relish',
 'Moroccan Mint Roasted Vegetables']

# Singular value decomposition


In [13]:
svd = TruncatedSVD(n_components=100, n_iter=9, random_state=42)
X_svd = svd.fit_transform(X)

# Clustering 

In [14]:
# Train K-Means
number_of_clusters=120
model = KMeans(n_clusters=number_of_clusters)
model.fit(X_svd[100:])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=120, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [15]:
preds = model.predict(X_svd[-100:])

In [42]:
def printCluster(preds,Y,n):
    c = 0
    for i in range(len(preds)):
        if preds[i]==n:
            c += 1
            print(Y[i])
    print ("Total:", c)
    
def getClusterNames(preds,Y,n):
    res = []
    for i in range(len(preds)):
        if preds[i]==n:
            res.append(Y[i])
    return res
            
            
def getCentroid(cluster):
    return np.asarray([np.mean(c) for c in np.transpose(cluster)])

def getCentroidNutrition(cluster):
    return np.asarray([np.mean(c) for c in np.transpose(cluster[-22:])])

def getIndices(preds, idx):
    return [i for i in range(len(preds)) if preds[i] == idx]

def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

def pearson(x,y):
    return pearsonr(x,y)[0]

In [67]:
def findSimilarClusters(preds,prior,Y,X,thr=1.1, n=3,c=120):
    cPrior = getCentroid(X[getIndices(preds,prior)])
    clusters = []
    for i in range(c):
        cI = getCentroid(X[getIndices(preds,i)])
        d = pearson(cPrior,cI)
        if d > thr:
            clusters.append((getClusterNames(preds,Y,i),d))
    sorted(clusters, key=lambda x: x[1])
    return clusters[:n]            

In [90]:
def cluster2json(fileName,outFile,clusters):
    f = open(fileName,'r')
    o = open(outFile,'w+')
    j = json.load(f)
    d = {}
    for c in clusters:
        for cc in c[0]:
            d[cc] = j[cc]
    json.dump(d, o,ensure_ascii=False)
    o.close()
    f.close()

In [17]:
preds

array([ 35,  33,  25,  93,  12,  54,  34,  76,  84,  76,  84,  97,  49,
        97,  97, 105, 107,  93,  14,  97,  97,  12, 107,  33,  35,  71,
        11, 119,  59,  76, 107,  47,  61,  76,  14,  84,  45,  30,  97,
        14,  33,  97,  82, 111,  78,  71,  76,  84, 115,   7, 111, 111,
        97, 114,  86,  86,  71,  84,  80, 105,  66,   9,  76, 107,  12,
        61,  67,  25,  76,   7,  97,  97,  86,  12,  12,   7,  12,  76,
        84,  97, 107,  93, 112,  19, 104, 107, 107,  12,  71, 106,  61,
        12,  82,  76,  32,  86,  84,  97,  97,  84], dtype=int32)

In [18]:
#Hierachical clustering
#aggModel = AgglomerativeClustering(n_clusters=120, linkage='ward')
aggModel = pickle.load(open('aggClusterModel.pkl','rb'))
preds = aggModel.fit_predict(X_svd)

In [19]:
preds[236]

81

In [20]:
c10 = getCentroid(X_svd[getIndices(preds,10)])
c25 = getCentroid(X_svd[getIndices(preds,25)])
c39 = getCentroid(X_svd[getIndices(preds,39)])
c16 = getCentroid(X_svd[getIndices(preds,16)])
c24 = getCentroid(X_svd[getIndices(preds,24)])
c = getCentroid(X_svd[getIndices(preds,20)])

print(dist(c10,c25))
print(dist(c10,c16))
print(dist(c10,c39))
print(dist(c10,c24))
print(dist(c10,c))
print(dist(c,c39))

0.449451345785
0.516873117783
1.05036437734
2.4941222311
4.93094324146
4.69125493973


In [35]:
print(pearson(c10,c25))
print(pearson(c10,c16))
print(pearson(c10,c39))
print(pearson(c10,c24))
print(pearson(c10,c))
print(pearson(c,c39))

0.829966724504
0.791947426387
0.952059588096
0.743674150879
0.519232171314
0.432956428778


In [24]:
modelFile='aggClusterModel.pkl'
pickle.dump(aggModel, open(modelFile,'wb'))

In [72]:
prior = 10
desserts = findSimilarClusters(preds,prior,Y,X_svd,thr=0.95, n=2,c=120)
print(sum([len(i[0]) for i in desserts]))

1448


In [119]:
prior=1
second = findSimilarClusters(preds,prior,Y,X_svd,thr=0.93, n=30,c=120)
print(sum([len(i[0]) for i in first]))

2091


In [120]:
prior=5
first = findSimilarClusters(preds,prior,Y,X_svd,thr=0.92, n=50,c=120)
print(sum([len(i[0]) for i in second]))

2091


In [121]:
cluster2json('recipes.json','desserts.json',desserts)

In [122]:
cluster2json('recipes.json','firsts.json',first)

In [123]:
cluster2json('recipes.json','seconds.json',second)